# 讀檔

In [1]:
import pandas as pd
import os
import csv

In [2]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/TEPC'

read_file_doc = 'keyword_by_author_cluster_%s.csv'

write_file_to = '{0}/{1}'.format(filepath, 'tepc_20190711_v3.xlsx')

In [3]:
clusters = {1:[], 2:[], 3:[], 4:[], 5:[]}
for n in range(1, 6):
    fileToRead = read_file_doc % (n)
    readFile = '{0}/{1}'.format(filepath, fileToRead)
#     print(readFile)
    with open(readFile) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            if row[1] == 'AUTHOR':
                continue
            clusters[n].append(row[1])
        
# print(clusters)

# Query the Organization and Societal sector

In [4]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/all_org_classes'

v10_tana = 'output_class_to_orgs_20190501_v10_tana.xlsx'

read_v10_tana = '{0}/{1}'.format(filepath, v10_tana)

In [5]:
workDf = pd.read_excel(read_v10_tana)
workDf = workDf.fillna(0)
workDf.shape, workDf.head()

((19439, 5),
      Graph   Organization        SocietalSector                        Period  \
 0  chrj_pr  _台灣青年_刊物任發行編輯             MassMedia      EarlyPostWarOrganization   
 1  chrj_pr         艋舺保良分局  PublicAdministration     UnknownPeriodOrganization   
 2  chrj_pr         鹿港保良總局  PublicAdministration     UnknownPeriodOrganization   
 3  chrj_pr         林本源製糖社       PrivateBusiness  JapaneseColonialOrganization   
 4  chrj_pr            阿緱廳  PublicAdministration  JapaneseColonialOrganization   
 
   Correct name  
 0      台灣青年雜誌社  
 1            0  
 2            0  
 3            0  
 4            0  )

# SPARQL

In [6]:
import stardog
import json

adminFile = '/Users/vincent/Projects/TBIO/tbio-conn-admin-local.json'
conn_details = {}
with open(adminFile, 'r') as readFile:
    conn_details = json.loads(readFile.read())

In [7]:
def sparqlQryOrg(name):
    return """
SELECT DISTINCT ?nameVal ?orgVal ?yearVal ?graphVal WHERE {
    ?name ?p ?orgEvt .
    GRAPH ?graph {
        ?orgEvt ?orgp ?org .
    }
    ?org a tbio:Organization .
    FILTER(sameterm(?name, <http://tbio.orient.cas.cz#%s>)) .
    OPTIONAL{?orgEvt <http://tbio.orient.cas.cz#occursInTime> ?year}
    
    BIND(STR(?name) AS ?nameStr) .
    BIND(REPLACE(?nameStr, "http://tbio.orient.cas.cz#", "") AS ?nameVal) .
    BIND(STR(?org) AS ?orgStr) .
    BIND(REPLACE(?orgStr, "http://tbio.orient.cas.cz#", "") AS ?orgVal) .
    BIND(STR(?year) AS ?yearStr) .
    BIND(REPLACE(?yearStr, "http://tbio.orient.cas.cz#", "") AS ?yearVal) .
    BIND(STR(?graph) AS ?graphStr) .
    BIND(REPLACE(?graphStr, "http://tbio.orient.cas.cz/", "") AS ?graphVal) .
} ORDER BY (?yearVal)
    """ % (name)

orgList=[]
def getOrg(queryRes, name, n):
    for nameVal in queryRes['results']['bindings']:
        name = nameVal['nameVal']['value'] if 'nameVal' in nameVal else ""
        org = nameVal['orgVal']['value'] if 'orgVal' in nameVal else ""
        year = nameVal['yearVal']['value'] if 'yearVal' in nameVal else ""
        graph = nameVal['graphVal']['value'] if 'graphVal' in nameVal else ""
        
        if len(org) == 0:
            continue
        
        match = workDf.loc[workDf['Organization'] == org]
        if match.empty == False:
            SocietalSector = match.iloc[0]['SocietalSector']
        else:
            SocietalSector = 'None'
            
        if SocietalSector == 0:
            SocietalSector = 'None'
        
        row = [n, name, org, SocietalSector, year, graph]
#         print(row)
        
        if row not in orgList:
            orgList.append(row)

with stardog.Connection('tbio', **conn_details) as conn:
    for n in range(1, 6):
        for name in clusters[n]:
#             print(name)
            query = sparqlQryOrg(name)
            results = conn.select(query)
            getOrg(results, name, n)

orgList[0:1]
print(len(orgList))

287


# Output data

In [8]:
outDf = pd.DataFrame(orgList, columns=['Cluster', 'Name', 'Organization', 'Societal sector', 'Year', 'Graph'])
outDf.head()

,Cluster,Name,Organization,Societal sector,Year,Graph
0,1,吳瀛濤,臺灣藝術社,VoluntaryAssociation,,tww
1,1,吳瀛濤,臺灣藝術社,VoluntaryAssociation,,tww2
2,1,吳瀛濤,臺北帝國大學圖書館,Academia_Education,,tww
3,1,吳瀛濤,臺北帝國大學圖書館,Academia_Education,,tww2
4,1,吳瀛濤,職員,None,,tww2


# Write file

In [9]:
with pd.ExcelWriter(write_file_to) as writer:
    # write file
    outDf.to_excel(writer, "TEPC", index=False)
    
    writer.save()